[Tutorial Source](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/)

* **Problem Statement**: There are 4 locations (labeled by different letters), and our job is to pick up the passenger at once location and drop them off at another. We receive +20 points for a successful drop-off and lose 1 point for every time step it takes. There is also a 10 point penalty for illegal pickup and dropoff actions.

## Imports and Setup

In [1]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



* The filled square represents the taxi, which is yellow without a passenger and green with a passenger
* The pipe represents a wall which the taxi cannot cross
* R, G, Y, and B are the possible pickup and dropoff locations. The blue letter is the current passenger pickup location and the purple letter is the current destination.


* **State Space**: The set of all possible situations the agent can inhabit (**5x5** possible taxi locations x **4** pickup/dropoff locations x **5** passenger states = **500 total states**)
* **Action Space**: The set of all the actions that agent can take in a given state (south, north, east, west, pickup, dropoff = **6 total actions**)

In [2]:
env.reset()
env.render()

print('Action space {}'.format(env.action_space))
print('State space {}'.format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action space Discrete(6)
State space Discrete(500)


## Solving environment without Reinforcement Learning

In [3]:
env.s = 328  # set environment to a random state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 4735
Penalties incurred: 1505


In [4]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 4735
State: 0
Action: 5
Reward: 20


## Solving environment with Reinforcement Learning

**Q-learning**
* A Q-value for a state-action combination is representative of the "quality" of an action taken from that state. Better Q-values imply better chances of getting greater rewards. For all possible actions, select the one with the highest Q-value.
* Q-values are updating using the equation:
$$ Q(state, action) \leftarrow (1 - \alpha)Q(state, action) + \alpha(\textrm{reward} + \gamma maxQ(next state, all actions)$$

  where $\alpha$ is the learning rate and $\gamma$ is the discount factor, which determines how much importance is given to future rewards (higher value captures more of a long-term reward)
* Learning the proper action to take in the current state by looking at the reward for the current state-action combination, and the maximum rewards for the next state.


In [5]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [6]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action)
        
        print(next_state)
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 73600
121
21
21
1
17
117
217
237
337
237
257
277
297
197
97
85
251
231
211
311
411
419
319
219
219
239
259
279
379
479
475
127
227
247
147
167
187
87
87
99
79
179
279
279
379
479
475
221
321
221
121
21
1
17
117
217
237
337
237
257
357
257
157
177
197
97
197
97
85
274
374
474
478
378
278
258
238
218
318
418
418
410
309
409
417
317
217
237
257
277
297
197
97
97
85
262
242
222
122
102
2
18
118
218
318
418
410
194
294
274
174
274
374
474
478
378
278
258
238
218
318
418
410
472
476
376
276
256
236
216
116
16
0
187
87
99
199
179
279
379
479
475
261
241
221
121
21
1
17
117
217
237
257
277
297
197
197
97
85
181
281
261
261
241
221
121
21
1
17
117
217
237
257
277
297
197
97
85
87
99
199
179
279
379
479
475
354
254
274
374
474
474
478
378
278
258
238
218
318
318
418
410
229
229
209
309
409
417
317
217
237
257
277
297
197
97
85
232
252
272
292
272
372
372
472
476
376
276
256
236
216
116
16
0
108
208
228
208
308
408
416
316
216
116
16
0
186
86
98
86
98
198
178
278
258
158
258
238
218
318


KeyboardInterrupt: 

In [7]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.93
Average penalties per episode: 0.0
